In [1]:
!pip install "numpy<2.0" grakel
import numpy as np, grakel
print(np.__version__)  # should show 1.26.x

1.26.4


In [4]:
from grakel.datasets import fetch_dataset
from grakel.utils import cross_validate_Kfold_SVM
from grakel.kernels import WeisfeilerLehman, VertexHistogram
import  os

In [5]:
# Path where you extracted MUTAG.zip
data_home = os.path.expanduser("~/Downloads")   # adjust if needed

# Ensure it’s extracted: you should see ~/Downloads/MUTAG/MUTAG_A.txt etc.
MUTAG = fetch_dataset("MUTAG", data_home=data_home, download_if_missing=False, verbose=True)

G, y = MUTAG.data, MUTAG.target
print("Loaded", len(G), "graphs")
print("First graph:", G[0])
print("First label:", y[0])


Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Loaded 188 graphs
First graph: [{(3, 4), (4, 3), (5, 4), (12, 13), (5, 7), (14, 13), (8, 9), (9, 8), (9, 14), (17, 15), (10, 9), (1, 6), (13, 14), (6, 5), (15, 17), (4, 5), (14, 9), (5, 6), (9, 10), (1, 2), (10, 11), (2, 1), (11, 10), (6, 1), (15, 13), (15, 16), (13, 15), (16, 15), (3, 2), (12, 11), (4, 10), (8, 7), (10, 4), (2, 3), (11, 12), (13, 12), (7, 5), (7, 8)}, {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 1, 16: 2, 17: 2}, {(2, 1): 0, (1, 2): 0, (3, 2): 0, (2, 3): 0, (4, 3): 0, (3, 4): 0, (5, 4): 0, (4, 5): 0, (6, 5): 0, (5, 6): 0, (6, 1): 0, (1, 6): 0, (7, 5): 0, (5, 7): 0, (8, 7): 0, (7, 8): 0, (9, 8): 0, (8, 9): 0, (10, 9): 0, (9, 10): 0, (10, 4): 0, (4, 10): 0, (11, 10): 0, (10, 11): 0, (12, 11): 0, (11, 12): 0, (13, 12): 0, (12, 13): 0, (14, 13): 0, (13, 14): 0, (14, 9): 0, (9, 1

In [6]:
# Generates a list of kernel matrices using the Weisfeiler-Lehman subtree kernel
# Each kernel matrix is generated by setting the number of iterations of the
# kernel to a different value (from 2 to 7)
Ks = list()
for i in range(1, 7):
    gk = WeisfeilerLehman(n_iter=i, base_graph_kernel=VertexHistogram, normalize=True)
    K = gk.fit_transform(G)
    Ks.append(K)


# Performs 10-fold cross-validation over different kernels and the parameter C of
# SVM and repeats the experiment 10 times with different folds
accs = cross_validate_Kfold_SVM([Ks], y, n_iter=10)
print("Average accuracy:", str(round(np.mean(accs[0])*100, 2)) + "%")
print("Standard deviation:", str(round(np.std(accs[0])*100, 2)) + "%")

Average accuracy: 82.71%
Standard deviation: 2.89%


In [8]:
import pandas as pd
from pathlib import Path

In [22]:
loop8_data = pd.read_csv(Path("/home/gabriele/Documents/GitHub/ML-correlator/Graph_Edge_Data/den_graph_data_8.csv"))

In [23]:
loop8_data

,COEFFICIENTS,EDGES
0,1,"[(1, 6), (1, 7), (1, 11), (1, 12), (2, 5), (2,..."
1,1,"[(1, 8), (1, 9), (1, 11), (1, 12), (2, 7), (2,..."
2,1,"[(1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (2..."
3,1,"[(1, 4), (1, 5), (1, 6), (1, 12), (2, 3), (2, ..."
4,1,"[(1, 4), (1, 8), (1, 9), (1, 12), (2, 5), (2, ..."
...,...,...
1427,0,"[(1, 5), (1, 6), (1, 11), (1, 12), (2, 3), (2,..."
1428,0,"[(1, 4), (1, 5), (1, 8), (1, 9), (2, 3), (2, 6..."
1429,0,"[(1, 4), (1, 7), (1, 8), (1, 12), (2, 3), (2, ..."
1430,0,"[(1, 4), (1, 7), (1, 8), (1, 12), (2, 3), (2, ..."


In [24]:
# --- Data ---
data = loop8_data
data_cols = [col for col in data.columns if 'COEFFICIENTS' not in col]
target_col = 'COEFFICIENTS'
X = data[data_cols].values
y = data[target_col].values.ravel()

In [25]:
X

array([['[(1, 6), (1, 7), (1, 11), (1, 12), (2, 5), (2, 8), (2, 11), (2, 12), (3, 4), (3, 6), (3, 7), (3, 9), (3, 10), (4, 5), (4, 8), (4, 9), (4, 10), (5, 8), (5, 10), (5, 12), (6, 7), (6, 10), (6, 12), (7, 9), (7, 11), (8, 9), (8, 11), (9, 11), (10, 12), (11, 12)]'],
       ['[(1, 8), (1, 9), (1, 11), (1, 12), (2, 7), (2, 10), (2, 11), (2, 12), (3, 4), (3, 5), (3, 6), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 10), (5, 9), (5, 10), (5, 11), (6, 7), (6, 8), (6, 12), (7, 10), (7, 12), (8, 9), (8, 12), (9, 11), (10, 11)]'],
       ['[(1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 6), (3, 7), (3, 11), (3, 12), (4, 5), (4, 7), (4, 8), (4, 10), (5, 6), (5, 8), (5, 9), (6, 9), (6, 12), (7, 10), (7, 11), (8, 9), (8, 10), (9, 12), (10, 11), (11, 12)]'],
       ...,
       ['[(1, 4), (1, 7), (1, 8), (1, 12), (2, 3), (2, 5), (2, 6), (2, 10), (3, 5), (3, 6), (3, 9), (4, 7), (4, 8), (4, 11), (5, 9), (5, 10), (6, 9), (6, 10), (7, 11), (7, 12), (8, 11), (8, 

In [26]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [31]:
import networkx as nx
from grakel import Graph

In [32]:
def nx_to_grakel(G: nx.Graph):
    # nodes labeled by their degree as an example; replace with your labels if you have them
    nl = {v: {"label": G.degree[v]} for v in G.nodes()}
    # grakel expects (node_labels, edge_list) or dict-of-dict-like
    return Graph(initialization_object=G, node_labels={v: nl[v]["label"] for v in G.nodes()})

In [38]:
import ast

In [ ]:


# Import the helper from your ML‑correlator module
# Make sure that ML-correlator/GNN is on your PYTHONPATH or run from repo root.
from GNN.load_features import load_graph_structure

def nx_to_grakel(G: nx.Graph):
    """
    Convert a NetworkX graph to a GraKeL Graph object.
    Node labels default to the node degrees; change this if you have categorical node labels.
    """
    node_labels = {v: G.degree[v] for v in G.nodes()}
    return Graph(initialization_object=G, node_labels=node_labels)

def load_loop_graphs_grakel(loop_order: int, data_dir: Path = Path("../Graph_Edge_Data")):
    """
    Use load_graph_structure from load_features.py to load edge lists,
    then convert each graph to a GraKeL Graph.
    
    Returns:
        A list of GraKeL Graphs, one for each graph in the dataset.
    """
    graph_infos = load_graph_structure(loop_order, data_dir=data_dir)
    graphs = []
    for info in graph_infos:
        G = nx.Graph()
        # add nodes 0..num_nodes-1
        G.add_nodes_from(range(info["num_nodes"]))
        # edge_list from load_graph_structure contains bidirectional edges; adding duplicates is fine
        G.add_edges_from(info["edge_list"])
        # optionally preserve original node labels
        labels = {i: info["node_labels"][i] for i in range(info["num_nodes"])}
        graphs.append(Graph(initialization_object=G, node_labels=labels))
    return graphs

def load_loop_graphs_from_dataframe(df: pd.DataFrame,
                                    edges_col: str = "EDGES",
                                    label_col: str = "COEFFICIENTS"):
    """
    Convert a DataFrame with an 'EDGES' column storing edge lists as strings
    into GraKeL Graph objects.  Returns (graphs, labels).
    """
    graphs = []
    for edge_str in df[edges_col]:
        # safely evaluate the string representation of the edge list
        edges = ast.literal_eval(edge_str)
        G = nx.Graph()
        G.add_edges_from(edges)
        graphs.append(nx_to_grakel(G))
    labels = df[label_col].tolist()
    return graphs, labels

# ------------------------------------------------------------
# Example usage:

# Option A: use load_graph_structure to load the 8‑loop graphs
# graphs_8 = load_loop_graphs_grakel(loop_order=8, data_dir=Path("/path/to/Graph_Edge_Data"))

# Option B: load from a CSV/DataFrame with stringified edge lists
# loop8_df = pd.read_csv(
#     "/home/gabriele/Documents/GitHub/ML-correlator/Tree classifier for graphs/mixed_loops/features_tabular/8loopfeats.csv"
# )
# graphs_8_from_df, y_8 = load_loop_graphs_from_dataframe(loop8_df)

# Each entry in graphs_8 or graphs_8_from_df is a GraKeL Graph ready for use with GraKeL kernels.


In [ ]:
# Generates a list of kernel matrices using the Weisfeiler-Lehman subtree kernel
# Each kernel matrix is generated by setting the number of iterations of the
# kernel to a different value (from 2 to 7)
Ks = list()
for i in range(1, 7):
    gk = WeisfeilerLehman(n_iter=i, base_graph_kernel=VertexHistogram, normalize=True)
    K = gk.fit_transform(G)
    Ks.append(K)


# Performs 10-fold cross-validation over different kernels and the parameter C of
# SVM and repeats the experiment 10 times with different folds
accs = cross_validate_Kfold_SVM([Ks], y, n_iter=10)
print("Average accuracy:", str(round(np.mean(accs[0])*100, 2)) + "%")
print("Standard deviation:", str(round(np.std(accs[0])*100, 2)) + "%")